In [3]:
import sqlite3
from datetime import datetime, timedelta
import random

class Database:
    def __init__(self, db_name):
        self.conn = sqlite3.connect(db_name)
        self.cursor = self.conn.cursor()

    def create_tables(self):
        self.cursor.execute('''
            CREATE TABLE IF NOT EXISTS Manufacturers (
                manufacturer_id INTEGER PRIMARY KEY AUTOINCREMENT,
                name TEXT NOT NULL,
                country TEXT
            )
        ''')

        self.cursor.execute('''
            CREATE TABLE IF NOT EXISTS Suppliers (
                supplier_id INTEGER PRIMARY KEY AUTOINCREMENT,
                name TEXT NOT NULL,
                address TEXT
            )
        ''')

        self.cursor.execute('''
            CREATE TABLE IF NOT EXISTS Products (
                product_id INTEGER PRIMARY KEY AUTOINCREMENT,
                name TEXT NOT NULL,
                manufacturer_id INTEGER,
                supplier_id INTEGER,
                release_year INTEGER,
                FOREIGN KEY (manufacturer_id) REFERENCES Manufacturers (manufacturer_id),
                FOREIGN KEY (supplier_id) REFERENCES Suppliers (supplier_id)
            )
        ''')

        self.cursor.execute('''
            CREATE TABLE IF NOT EXISTS Customers (
                customer_id INTEGER PRIMARY KEY AUTOINCREMENT,
                name TEXT NOT NULL,
                address TEXT,
                phone_number TEXT
            )
        ''')

        self.cursor.execute('''
            CREATE TABLE IF NOT EXISTS Purchases (
                purchase_id INTEGER PRIMARY KEY AUTOINCREMENT,
                product_id INTEGER,
                customer_id INTEGER,
                purchase_date DATE,
                warranty_end_date DATE,
                FOREIGN KEY (product_id) REFERENCES Products (product_id),
                FOREIGN KEY (customer_id) REFERENCES Customers (customer_id)
            )
        ''')

        self.conn.commit()

    def insert_sample_data(self):
        # Вставка данных в таблицу Manufacturers
        manufacturers = [('Samsung', 'Корея'), ('Apple', 'США'), ('Xiaomi', 'Китай')]
        self.cursor.executemany('INSERT INTO Manufacturers (name, country) VALUES (?, ?)', manufacturers)

        # Вставка данных в таблицу Suppliers
        suppliers = [('ElectroWorld', 'New York'), ('TechLand', 'Los Angeles'), ('GadgetHub', 'Chicago')]
        self.cursor.executemany('INSERT INTO Suppliers (name, address) VALUES (?, ?)', suppliers)

        # Вставка данных в таблицу Products
        products = [('Galaxy S21', 1, 1, 2021), ('iPhone 13', 2, 2, 2021), ('Mi 11', 3, 3, 2021)]
        self.cursor.executemany('INSERT INTO Products (name, manufacturer_id, supplier_id, release_year) VALUES (?, ?, ?, ?)', products)

        # Вставка данных в таблицу Customers
        customers = [('Илья Зуев', 'New York', '7234567890'), ('Дзебан Арсений', 'Los Angeles', '7987654321')]
        self.cursor.executemany('INSERT INTO Customers (name, address, phone_number) VALUES (?, ?, ?)', customers)

        # Вставка данных в таблицу Purchases
        for i in range(1, 4):
            purchase_date = datetime.now() - timedelta(days=random.randint(1, 30))
            warranty_end_date = purchase_date + timedelta(days=random.randint(365, 730))
            self.cursor.execute('INSERT INTO Purchases (product_id, customer_id, purchase_date, warranty_end_date) VALUES (?, ?, ?, ?)',
                                (i, random.randint(1, 2), purchase_date.strftime('%Y-%m-%d'), warranty_end_date.strftime('%Y-%m-%d')))

        self.conn.commit()

    def get_products_with_manufacturers_and_suppliers(self):
        self.cursor.execute('''
            SELECT Products.name, Manufacturers.name, Suppliers.name
            FROM Products
            JOIN Manufacturers ON Products.manufacturer_id = Manufacturers.manufacturer_id
            JOIN Suppliers ON Products.supplier_id = Suppliers.supplier_id
        ''')
        return self.cursor.fetchall()

    def get_purchases_with_products_and_customers(self):
        self.cursor.execute('''
            SELECT Products.name, Customers.name, Purchases.purchase_date, Purchases.warranty_end_date
            FROM Purchases
            JOIN Products ON Purchases.product_id = Products.product_id
            JOIN Customers ON Purchases.customer_id = Customers.customer_id
        ''')
        return self.cursor.fetchall()

    def close(self):
        self.conn.close()


In [4]:
db = Database('electronics_store.db')
db.create_tables()
db.insert_sample_data()

print("Товары с производителями и поставщиками:")
for product in db.get_products_with_manufacturers_and_suppliers():
    print(product)

print("\nПокупки с товарами и клиентами:")
for purchase in db.get_purchases_with_products_and_customers():
    print(purchase)

db.close()

Товары с производителями и поставщиками:
('Galaxy S21', 'Samsung', 'ElectroWorld')
('iPhone 13', 'Apple', 'TechLand')
('Mi 11', 'Xiaomi', 'GadgetHub')
('Galaxy S21', 'Samsung', 'ElectroWorld')
('iPhone 13', 'Apple', 'TechLand')
('Mi 11', 'Xiaomi', 'GadgetHub')

Покупки с товарами и клиентами:
('Galaxy S21', 'Jane Smith', '2024-12-10', '2026-05-23')
('iPhone 13', 'Jane Smith', '2024-12-19', '2026-01-20')
('Mi 11', 'Jane Smith', '2024-12-24', '2026-06-06')
('Galaxy S21', 'John Doe', '2024-12-10', '2026-09-06')
('iPhone 13', 'Jane Smith', '2024-12-23', '2026-02-08')
('Mi 11', 'John Doe', '2024-12-23', '2026-01-20')
